In [2]:
import numpy as np
import torch
import torch.optim as optim
import math
import gym
from gym import spaces, logger
from gym.utils import seeding


class RigidEnv(gym.Env):
    """
    Source:
        This file is created by Nathan Lambert, adapted from a model from Somil Bansal
    Observation: 
        Type: Box(12)
        Num	Observation                 Min         Max
        0	x-pos                       -10         10      (meters)
        1	y-pos                       -10         10      (meters)
        2	z-pos                       -10         10      (meters)
        3	x-vel                       -Inf        Inf     (meters/sec)
        4   y-vel                       -Inf        Inf     (meters/sec)
        5   z-vel                       -Inf        Inf     (meters/sec)
        6   roll                        -180        180     (degrees)
        7   pitch                       -90         90      (degrees)
        8   yaw                         -180        180     (degrees)
        9   omega_x                     -Inf        Inf     (rad/s^2)
        10  omega_y                     -Inf        Inf     (rad/s^2)
        11  omega_z                     -Inf        Inf     (rad/s^2)
    """

    def __init__(self, dt=.01, x_noise=0.0, u_noise=0): #0.01 noise
        self.x_dim = 12
        self.u_dim = 4
        self.dt = dt
        self.x_noise = x_noise

        # simulate ten steps per return
        self.dt = self.dt

        # Setup the state indices
        self.idx_xyz = [0, 1, 2]
        self.idx_xyz_dot = [3, 4, 5]
        self.idx_ptp = [6, 7, 8]
        self.idx_ptp_dot = [9, 10, 11]

        self.seed()
        self.viewer = None
        self.state = None
        self.steps_beyond_done = None

    def seed(self, seed=None, inertial=False):
        self.np_random, seed = seeding.np_random(seed)
        if inertial:
            self.Ixx = self.Ixx * self.np_random.uniform(low=0.85, high=1.15, size=1)[0]
            self.Iyy = self.Iyy * self.np_random.uniform(low=0.85, high=1.15, size=1)[0]

        return [seed]

    def step(self, forces, torques):
        # assert self.action_space.contains(u), "%r (%s) invalid" % (u, type(u))

        # We need to convert from upright orientation to N-E-Down that the simulator runs in
        # For reference, a negative thrust of -mg/4 will keep the robot stable
        u = self.pwm_thrust_torque(pwm)
        state = self.state
        self.last_state = state

        dt = self.dt
        u0 = u
        x0 = state
        idx_xyz = self.idx_xyz
        idx_xyz_dot = self.idx_xyz_dot
        idx_ptp = self.idx_ptp
        idx_ptp_dot = self.idx_ptp_dot

        m = self.m
        L = self.L
        Ixx = self.Ixx
        Iyy = self.Iyy
        Izz = self.Izz
        g = self.g
        
        # Cross term, direct term for the three torques
        Tx = np.array([Iyy / Ixx - Izz / Ixx, L / Ixx])
        Ty = np.array([Izz / Iyy - Ixx / Iyy, L / Iyy])
        Tz = np.array([Ixx / Izz - Iyy / Izz, 1. / Izz])

        # # Add noise to input
        # u_noise_vec = np.random.normal(
        #     loc=0, scale=self.u_noise, size=(self.u_dim))
        # u = u+u_noise_vec

        # Array containing the forces
        # This assumes the force all comes directly out of the plane
        # Third column of Qb/i (converts body forces to inertial frame)
        Fxyz = np.zeros(3)
        Fxyz[0] = -1 * (math.cos(x0[idx_ptp[0]]) * math.sin(x0[idx_ptp[1]]) * math.cos(x0[idx_ptp[2]]) + math.sin(x0[idx_ptp[0]]) * math.sin(x0[idx_ptp[2]])) * u0[0] / m
        Fxyz[1] = -1 * (math.cos(x0[idx_ptp[0]]) * math.sin(x0[idx_ptp[1]]) * math.sin(x0[idx_ptp[2]]) - math.sin(x0[idx_ptp[0]]) * math.cos(x0[idx_ptp[2]])) * u0[0] / m
        Fxyz[2] = g - 1 * (math.cos(x0[idx_ptp[0]]) * math.cos(x0[idx_ptp[1]])) * u0[0] / m

        # Compute the torques
        t0 = np.array([x0[idx_ptp_dot[1]] * x0[idx_ptp_dot[2]], u0[1]])
        t1 = np.array([x0[idx_ptp_dot[0]] * x0[idx_ptp_dot[2]], u0[2]])
        t2 = np.array([x0[idx_ptp_dot[0]] * x0[idx_ptp_dot[1]], u0[3]])
        Txyz = np.array([Tx.dot(t0), Ty.dot(t1), Tz.dot(t2)])

        x1 = np.zeros(12)
        x1[idx_xyz_dot] = x0[idx_xyz_dot] + dt * Fxyz
        x1[idx_ptp_dot] = x0[idx_ptp_dot] + dt * Txyz
        x1[idx_xyz] = x0[idx_xyz] + dt * x0[idx_xyz_dot]
        x1[idx_ptp] = x0[idx_ptp] + dt * self.pqr2rpy(x0[idx_ptp], x0[idx_ptp_dot])

        # makes states less than 1e-12 = 0
        x1[abs(x1) < 1e-15] = 0
        self.state = x1
        state = x1

        # Add noise component
        x_noise_vec = np.random.normal(
            loc=0, scale=self.x_noise, size=(self.x_dim))

        self.state += x_noise_vec

        obs = self.get_obs()
        reward = self.get_reward(obs, u)
        done = self.get_done(obs)

        return obs, reward, done, {}

    def get_obs(self):
        raise NotImplementedError("Subclass must implement this function")

    def set_state(self, x):
        self.state = x

    def reset(self, safe=False):
        x0 = np.array([0, 0, 0])
        v0 = self.np_random.uniform(low=-0.01, high=0.01, size=(3,))
        # ypr0 = self.np_random.uniform(low=-0.0, high=0.0, size=(3,))
        if safe:
            ypr0 = self.np_random.uniform(low=-0.01, high=0.01, size=(3,))
            ypr0 = self.np_random.uniform(low=-0.00, high=0.00, size=(3,))
            w0 = self.np_random.uniform(low=-0.00, high=0.00, size=(3,))
        else:
            ypr0 = self.np_random.uniform(low=-np.pi/16., high=np.pi/16., size=(3,))
            w0 = self.np_random.uniform(low=-0.01, high=0.01, size=(3,))

        ypr0[-1] = 0 # 0 out yaw

        self.state = np.concatenate([x0, v0, ypr0, w0])
        self.last_state = self.state
        self.steps_beyond_done = None
        self.interal = 0

        return self.get_obs()

    def get_done(self, state):
        # Done is pitch or roll > 35 deg
        max_a = np.deg2rad(45)
        d = (abs(state[1]) > max_a) or (abs(state[0]) > max_a)
        return d

    def pqr2rpy(self, x0, pqr):
        rotn_matrix = np.array([[1., math.sin(x0[0]) * math.tan(x0[1]), math.cos(x0[0]) * math.tan(x0[1])],
                                [0., math.cos(x0[0]), -math.sin(x0[0])],
                                [0., math.sin(x0[0]) / math.cos(x0[1]), math.cos(x0[0]) / math.cos(x0[1])]])
        return rotn_matrix.dot(pqr)




In [75]:
# Set sun, earth position
sun_mass = 1.98892 * 10**30
earth_mass = 5.9742 * 10**24
sun_to_earth = 1 # AU, check to make sure gravity is in AU

# Functions for computing gravity vectors
Gsc = 1.361 # kW/m2
A = 1 # m**2     # sail area
C_SRP    = 2.00 # SRP coefficent, C_SRP = 1 for perfectly absorbing, C_SRP = 2 for perfectly reflective  solar sail
# FN= 2GsccAcos2(α)≈(9μN/m2) cos2(α)

dim = 2 # 2 or 3d sim, starting with 2

# The gravitational constant G
G = 6.67428e-11

# Assumed scale: 100 pixels = 1AU.
AU = (149.6e6 * 1000)     # 149.6 million km, in meters.
SCALE = 250 / AU

# constants for inertial properties and torques
global r_ss, rp_ss, A_ss, Ap_ss, hp_ss, lp_ss, P_ss
r_ss = 0.56 #m, 
rp_ss = 0.565 #m
A_ss = 1 #m**2, 
Ap_ss = 0.02 #m**2
hp_ss = 10E-2 #m  
lp_ss = 20E-2 #m.
m_mp = 0.01 # kg (1gram)
m_ss = 0.0016 # kg (1.6grams)
mp_ss = 0.000032 # kg (32mg) 
P_ss = 9E-6 # uPa. (estimate)

# equations for roll flag control
# thetap_r = 2*math.asin(dyp_r/2/lp_ss)
# alphap_r = alpha - thetap_r

# Fp_s = f(alpha_roll, theta) = P_ss*Ap_ss*cos(alphap_r)**3
def Fp_s(alpha_r):
    return P_ss*Ap_ss*(math.cos(alpha_r)**2)

# F_s = f(alpha)  = P_ss*A_ss* cos(alpha)**3
def F_s(alpha_r):
    return P_ss*A_ss*(math.cos(alpha_r)**2)

# dx,dz in 0-10mm
#def T_yy(angle):
    # T_yy = Fp_s * rp_ss * math.sin(thetap_r) # = I_yy*theta_dot_dot

def T_zz(angle, dx):
    # T_zz = F_s*dx #  I_zz*alpha_dot_dot
    return F_s(angle)*dx

def T_xx(angle, dz):
    # T_xx = F_s*dz #  I_xx*alpha_dot_dot
    return F_s(angle)*dz

def inertials(dx,dy,dz,m_mp = 0.01, m_ss = 0.0016):
    ratio = m_mp/(m_ss+m_mp)
    x_ss = dx*ratio
    x_mp = dx*(1-ratio)
#     y_ss = dy*ratio
    y_ss = 2*ratio
#     y_mp = dy*(1-ratio)
    y_mp = 2*(1-ratio)
    z_ss = dz*ratio
    z_mp = dz*(1-ratio)

    I_yy = 1 / 2 * m_ss * r_ss **2+ m_ss * (x_ss **2+ z_ss **2)+mp_ss * rp_ss **2+1 / 12 * mp_ss * hp_ss **2 + m_mp * (x_mp **2+z_mp**2)
    I_zz = 1 / 4 * m_ss * r_ss **2+m_ss * (x_ss **2+ y_ss **2)+m_mp * (x_mp**2+y_mp **2)
    I_xx = 1 / 4 * m_ss * r_ss **2+m_ss * (z_ss **2+ y_ss **2)+m_mp * (z_mp**2+y_mp **2)
    return I_xx, I_yy, I_zz

f = F_s(0)
n = 100
alpha = np.linspace(-np.pi/2,np.pi/2,n)
dx = np.linspace(-0.01,0.01,n)
dz = np.linspace(-0.01,0.01,n)
xx,zz = np.meshgrid(dx,dz)
rates = np.zeros((n,n,3))
for i, x in enumerate(dx):
    for j, a in enumerate(alpha):
        f = F_s(a)
        ix, iy, iz = inertials(x,0,z)
        z = x
        tx = f *z
        ty = 0
        tz = f* x
        ox = tz/ix
        oy = 0
        oz = tz/iz
        rates[i,j,0] = ox
        rates[i,j,2] = oz 

        
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
        z=rates[:,:,0],
        x=alpha,
        y=dx,
        colorscale='Viridis')
                )

fig.update_layout(
    title='Angular Rates vs Sun Angle vs Input',
    xaxis_title='cone angle (rad)',
    yaxis_title='input motion (m)')

fig.show()

In [ ]:

class SolarSail
    def __init__(self, mass, Ixx, Iyy, Izz):
        self.mass = mass
        self.Ixx = Ixx
        self.Iyy = Iyy
        self.Izz = Izz
        
    def pos2force(self, cone, clock=None):    
        # FN= 2GsccAcos2(α)≈(9μN/m2) cos2(α)
        # RIGHT NOW FORCE IS AT CENTER OF MASS
        # SRP on Sail (A_SRP is a constant 1.0 m**2 right now but through control of
        # sail orientation, we can have this vary between 0 <= A_SRP <= 1.
        # avec_SRP_Sail     = (1/m) * P_SRP(r_Sun_Sail) * C_SRP * A_SRP * rvec_Sun_Sail/r_Sun_Sail;

        
    
    def cone2torques(self, cone, clock=None):

    def gravity(self, obj):
        # Compute the distance of the other body.
        sx, sy = self.px, self.py
        ox, oy = other.px, other.py
        dx = (ox-sx)
        dy = (oy-sy)
        d = math.sqrt(dx**2 + dy**2)

        # Report an error if the distance is zero; otherwise wepll
        # get a ZeroDivisionError exception further down.
        if d == 0:
            raise ValueError("Collision between objects %r and %r"
                             % (self.name, other.name))

        # Compute the force of attraction
        f = G * self.mass * other.mass / (d**2)

        # Compute the direction of the force.
        theta = math.atan2(dy, dx)
        fx = math.cos(theta) * f
        fy = math.sin(theta) * f
        return fx, fy


In [ ]:
# PID controller for roll (add noise to simulation)


In [ ]:
# loop of computing forward dynamics

# need info about initial condition of orbit
v = sqrt(GM/r)

# low orbit: 180-20000 km
# mid orbit: 2000-35780 km
# high orbit: >35,780 km

https://en.wikipedia.org/wiki/Orbital_speed
    

In [8]:
# visualize
# from https://medium.com/analytics-vidhya/simulating-the-solar-system-with-under-100-lines-of-python-code-5c53b3039fc6
import plotly
import plotly.graph_objects as go

e = [0,0]
r = [0, -6.684587e-5] # xy pos sail relative to earth, 10000km below in this diagram
s = [-1, 0] # sun is 1 AU to the "left" of earth

mu = 3.986004418*10**14 # SGP earth

t_vec = np.linspace(0,100000, 1000)
v0_sail = [np.sqrt(mu,r[0]),0]

def pos2sail(position):
    # function for generating sail angle from position of sun
    return 0

for t in t_vec:
    f_ss = 
    f_ge =
    f_gs = 
# Create figure
fig = go.Figure(
    data=[go.Scatter(x=x, y=y,
                     mode="lines",
                     line=dict(width=2, color="blue")),
          go.Scatter(x=x, y=y,
                     mode="lines",
                     line=dict(width=2, color="blue"))],
    layout=go.Layout(
        xaxis=dict(range=[xm, xM], autorange=False, zeroline=False),
        yaxis=dict(range=[ym, yM], autorange=False, zeroline=False),
        title_text="Kinematic Generation of a Planar Curve", hovermode="closest",
        updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None])])]),
    frames=[go.Frame(
        data=[go.Scatter(
            x=[xx[k]],
            y=[yy[k]],
            mode="markers",
            marker=dict(color="red", size=10))])

        for k in range(N)]
)

fig.show()

NameError: name 'x' is not defined